<a href="https://colab.research.google.com/github/arbin34/heruk/blob/main/longchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries for environment handling and OpenAI API integration
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

# Load environment variables from .env file to securely access the OpenAI API key
load_dotenv()

# Initialize OpenAI model with the API key and set creativity level (temperature)
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.7)

# Define a function to generate a restaurant name and menu items based on cuisine
def generate_restaurant_name_and_items(cuisine):
    # Chain 1: Create a prompt to generate a fancy restaurant name based on the cuisine
    prompt_template_name = PromptTemplate(
        input_variables=['cuisine'],
        template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
    )
    name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

    # Chain 2: Create a prompt to suggest menu items for the generated restaurant name
    prompt_template_items = PromptTemplate(
        input_variables=['restaurant_name'],
        template="Suggest some menu items for {restaurant_name}. Return it as a comma separated string"
    )
    food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

    # Combine both chains to sequentially generate restaurant name and menu items
    chain = SequentialChain(
        chains=[name_chain, food_items_chain],
        input_variables=['cuisine'],
        output_variables=['restaurant_name', "menu_items"]
    )

    # Execute the chain with the specified cuisine and return the response
    response = chain({'cuisine': cuisine})
    return response

# Main execution block to run the function and print results for Italian cuisine
if __name__ == "__main__":
    print(generate_restaurant_name_and_items("Italian"))

# Import Streamlit for building the web interface
import streamlit as st
import langchain_helper

# Set up the title for the Streamlit app
st.title("Restaurant Name Generator")

# Sidebar for user to select a cuisine type
cuisine = st.sidebar.selectbox("Pick a Cuisine", ("Indian", "Italian", "Mexican", "Arabic", "American"))

# If a cuisine is selected, generate and display the restaurant name and menu items
if cuisine:
    response = langchain_helper.generate_restaurant_name_and_items(cuisine)
    st.header(response['restaurant_name'].strip())  # Display the restaurant name
    menu_items = response['menu_items'].strip().split(",")  # Parse and split menu items
    st.write("**Menu Items**")
    for item in menu_items:
        st.write("-", item)  # Display each menu item
